In [ ]:
#import de toutes les librairies usuelles au projets
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from google.colab import drive

In [ ]:
#recuperation du dataset depuis mom google drive
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/tropics/Train.zip
!unzip gdrive/MyDrive/tropics/Test.zip
# determination du nombre de classe de la sortie
folders = glob('./Train/*')


Mounted at /content/gdrive
Archive:  gdrive/MyDrive/tropics/Train.zip
replace Train/Parasite/C189P150ThinF_IMG_20151203_142113_cell_91.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
folders

In [ ]:
# Créer un modele cnn
# Ceci est un modèle personnalisé utilisant CNN, nous pouvons utiliser ce modèle au lieu de VGG9 si nous le voulons
# La précision peut différer dans les deux modèles.

model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.summary()

In [ ]:
# indication de la fonction cout et de la methode d'optimisation
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# générateur de données d'image pour importer les images du jeu de données
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# fournir la même taille cible que celle initialisée pour la taille de l'image
training_set = train_datagen.flow_from_directory('./Train',
                                                 target_size = (127, 124),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('./Test',
                                            target_size = (127, 124),
                                            batch_size = 32,
                                            class_mode = 'categorical')


In [ ]:
# entrainer le modele
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
training_set.class_indices

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#enregistrer le modele 

model.save('tropics.h5')

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(
model, to_file='model.png', show_shapes=True)

In [ ]:
!unzip gdrive/MyDrive/tropics/Test2.zip

In [ ]:
test_set2 = test_datagen.flow_from_directory('./Test2',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
print(y_pred.shape)
#y_pred = np.argmax(Y_pred, axis=1) - used for multiclass
y_predi = (y_pred > 0.5) * 1.0
y_val=["Malaria" "Health"]
 
y_predi.sum()

In [ ]:
img=image.load_img('./Test/Uninfected/C3thin_original_IMG_20150608_162922_cell_191.png',target_size=(224,224))
img

In [ ]:
x=image.img_to_array(img)

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
if(a==0):
    print("Uninfected")
else:
    print("Infected")

In [ ]:
#enregistrer le modele 

model.save('tropics.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!pip3 install gradio

In [ ]:
model.save("model2")

In [ ]:
import gradio as gr
def sentence_builder(quantity, animal, place, activity_list, morning):
    return f"""The {quantity} {animal}s went to the {place} where they {" and ".join(activity_list)} until the {"morning" if morning else "night"}"""


iface = gr.Interface(
    sentence_builder,
    [
        gr.inputs.Slider(2, 20),
        gr.inputs.Dropdown(["cat", "dog", "bird"]),
        gr.inputs.Radio(["park", "zoo", "road"]),
        gr.inputs.CheckboxGroup(["ran", "swam", "ate", "slept"]),
        gr.inputs.Checkbox(label="Is it the morning?"),
    ],
    "text",
    examples=[
        [2, "cat", "park", ["ran", "swam"], True],
        [4, "dog", "zoo", ["ate", "swam"], False],
        [10, "bird", "road", ["ran"], False],
        [8, "cat", "zoo", ["ate"], True],
    ],
)


In [ ]:
folders

In [ ]:
folders

In [ ]:
!pip3 install opencv-python
import gradio as gr
import cv2
def greet(name):
    return "Hello " + name + "!!"



def image_mod(image):
    image = cv2.resize(image,(224,224),3)
    a=np.argmax(model.predict(image), axis=1)
    if(a==1):
        print("Uninfected")
    if(a==0):
        print("Infected")
    
    #iface = gr.Interface(fn=greet, inputs="text", outputs="text")
    #iface.launch()
    

    return image.rotate(45)
  


iface = gr.Interface(image_mod, gr.inputs.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label=None, optional=False), "image")
iface.launch()
